# Метрики precision и recall

Данные содержат признаки пользователей и целевую переменную affair - была ли связь на стороне

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor

In [2]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Превращаем строковые переменные в логические массивы
X_workclass=pd.get_dummies(data[['workclass']])
X_workclass=X_workclass.drop(['workclass_?'], axis=1)
X_education=pd.get_dummies(data[['education']])
X_marital=pd.get_dummies(data[['marital-status']])
X_occupation=pd.get_dummies(data[['occupation']])
X_relationship=pd.get_dummies(data[['relationship']])
X_race=pd.get_dummies(data[['race']])
X_gender=pd.get_dummies(data[['gender']])
X_country=pd.get_dummies(data[['native-country']])
X_country=X_country.drop(['native-country_?'], axis=1)
# Создаем массив X
X=data[['age','capital-gain','capital-loss','hours-per-week']]
X=pd.concat([X, X_workclass,X_education,X_marital,X_occupation,X_relationship,X_race,X_gender,X_country], axis=1, sort=False)

In [18]:
# Создаем массив y
y=data[['income']]
y[:]=0
y[data[['income']]=='>50K']=1

C:\Users\Volk\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Volk\Anaconda3\lib\site-packages\pandas\core\frame.py:3374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(key, value)
C:\Users\Volk\Anaconda3\lib\site-packages\pandas\core\frame.py:3362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [9]:
# делим выборку на tarin и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12354)

# нормируем массивы X_train и X_test
scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# уменьшаем размерность исходной выборки (В двое)
pca = PCA(n_components=round(X.shape[1]/2))
pca.fit(X_train)
X_pca_train=pca.transform(X_train)
X_pca_test=pca.transform(X_test)

C:\Users\Volk\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Volk\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
C:\Users\Volk\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.


In [31]:
# построим модель логической регрессии
model_lr = LogisticRegression()
model_lr.fit( X_pca_train, y_train )
y_pred_lr = model_lr.predict(X_pca_test)

#для данной модели посчитаем её ROC
roc_auc_score( y_test, y_pred_lr )

C:\Users\Volk\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Volk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7457567634971853

In [ ]:
for n in range(1,11):
    # построим множество моделей knn с разными n=1,10
    model_knn = KNeighborsRegressor(n_neighbors=n)
    model_knn.fit( X_pca_train, y_train )
    y_pred_knn = model_knn.predict(X_pca_test)
    # для каждой модели посчитаем её ROC
    # (причем результаты предсказания округлим т.к. knn выдает непрерывные значения)
    print([n,roc_auc_score( y_test, y_pred_knn.round() )])

[1, 0.721926150081114]
[2, 0.6673708353105439]
[3, 0.7373986678420889]
[4, 0.6999637346920214]
[5, 0.7386049435110912]
[6, 0.7202898131623752]


Логическая регрессия дает лучший результат knn для данной задачи и делает это быстрее. Для knn заметна тенденция к улучшению качества с ростом n.